In [ ]:
# default_exp scrape8K

# scrape8K

> Scrape item summaries from 8-K SEC filings.

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
from nbdev import show_doc

In [ ]:
#export

import collections
import itertools
import os
import re

from secscan import utils, dailyList, basicInfo, infoScraper

default8KDir = os.path.join(utils.stockDataRoot,'scraped8K')

8K scraper class - scrape items summary from the SEC filing:

In [ ]:
#export

itemPat = re.compile(r'item\s*(\d+(?:\.\d*)?)',re.IGNORECASE)
def parse8K(accNo, formType=None, textLimit=basicInfo.defaultTextLimit) :
    info = basicInfo.getSecFormInfo(accNo, formType=formType, get99=True, textLimit=textLimit)
    links = info['links']
    if len(links) == 0 :
        utils.printErrInfoOrAccessNo('NO LINKS LIST in',accNo)
        return info
    items = info.get('items',[])
    if len(items) == 0 :
        return info
    mainText = utils.downloadSecUrl(links[0][3], toFormat='souptext')
    itemPosL = [0]
    info['itemTexts'] = itemTexts = [None for item in items]
    for i,item in enumerate(items) :
        m = itemPat.match(item)
        if m is None :
            utils.printErrInfoOrAccessNo(f"unexpected format for item header {item}",accNo)
            continue
        m = re.search(r'item\s*'+m.group(1), mainText[itemPosL[-1]:], re.IGNORECASE)
        if m is None :
            utils.printErrInfoOrAccessNo(f"couldn't find {item}",accNo)
            continue
        itemPosL.append(itemPosL[-1]+m.start())
        itemTexts[i] = ''
        # print('pos for',item,itemPosL[-1])
    itemPosL.append(len(mainText))
    j = 1
    for i in range(len(itemTexts)) :
        if itemTexts[i] is None :
            itemTexts[i] = items[i] + ' ???'
        else :
            itemTexts[i] = mainText[itemPosL[j] : min(itemPosL[j]+textLimit, itemPosL[j+1])]
            j += 1
    return info
#     print(firstSLText)
#     print 'size',len(firstSLText)
#     print utils.secUrlPref+links[0][3]
#     info['itemsdetail'] = getSec8KItemsFromText(firstSLText)
#     if info['itemsdetail'] == [] :
#         print('NO ITEMS:',utils.secUrlPref+links[0][3])
#     else :
#         print('.', #info['itemsdetail'],
#               end=' ')
#     return info

secItemHeaderPatStrs = [
    r'item[^a-z]*?(\d+(?:\.\d*)?)\.(?!\d)',
    r'item[^a-z]*?(\d+(?:\.\d*)?)(?=[a-z\s]*[.:])',
]
secItemPats = [re.compile(secItemHeaderPatStr
                            + r'\s*(.{80,}?)('+secItemHeaderPatStr+'|$)',
                            re.IGNORECASE)
                for secItemHeaderPatStr in secItemHeaderPatStrs]

def getSecItemsFromTextUsingPat(txt, secItemPat, filterF=None ,maxHeadLen=80, maxBodyLen=500) :
    pos = 0
    res = []
    while True :
        m = secItemPat.search(txt,pos)
        if m is None :
            return res
        g = m.groups()
        if len(res)>0 and res[-1][0] == g[0] :
            pos = m.start()+1
        else :
            head = g[1][:maxHeadLen]
            #if head.find('.') >= 0 :
            #    head = head[:head.find('.')+1]
            newTup = (g[0],head,g[1][:maxBodyLen])
            pos = m.start(3)
            if (filterF is None) or filterF(newTup) :
                res.append(newTup)

def getSecItemsFromText(txt, filterF=None, maxHeadLen=80, maxBodyLen=500) :
    """
    Returns a list of item summaries for items from 8-K and SC 13D forms:
        format in form:
            Item 1.01. WWW. YYY ZZZ
            Item 3.01. AAA. BBB CCC
        returned value:
            [('1.01', 'WWW.', 'WWW. YYY ZZZ')
             ('3.01', 'AAA.', 'AAA. BBB CCC') ... ]
    """
    for secItemPat in secItemPats :
        res = getSecItemsFromTextUsingPat(txt,secItemPat,filterF,maxHeadLen,maxBodyLen)
        if res :
            break
    return res

financialStatementsPat = re.compile(r'[^a-z]*financial\s*statements',
                                    re.IGNORECASE)
def getSec8KItemsFromText(txt) :
    """
    Returns list of item summaries for 8-K forms, ignoring the
    'Financial Statements and Exhibits' item.
    """
    return getSecItemsFromText(txt, lambda item : not financialStatementsPat.match(item[1]))

class scraper8K(infoScraper.scraperBase) :
    def __init__(self, infoDir=default8KDir, startD=None, endD=None, fSuff='m.pkl', **pickle_kwargs) :
        super().__init__(infoDir, '8-K', startD=startD, endD=endD, fSuff=fSuff, **pickle_kwargs)
    def scrapeInfo(self, accNo, formType=None) :
        return parse8K(accNo, formType), None

Test 8-K scraper class:

In [ ]:
# parse8K('0000909108-21-000058','8-K')

{'links': [('dhil-20210728.htm',
   '8-K',
   '8-K',
   '/ix?doc=/Archives/edgar/data/909108/000090910821000058/dhil-20210728.htm'),
  ('dhil-20210630xexx991.htm',
   'EX-99.1',
   'EX-99.1',
   '/Archives/edgar/data/909108/000090910821000058/dhil-20210630xexx991.htm'),
  ('dhillogoa14a.jpg',
   '',
   'GRAPHIC',
   '/Archives/edgar/data/909108/000090910821000058/dhillogoa14a.jpg'),
  ('dhil-20210728.xsd',
   'XBRL TAXONOMY EXTENSION SCHEMA DOCUMENT',
   'EX-101.SCH',
   '/Archives/edgar/data/909108/000090910821000058/dhil-20210728.xsd'),
  ('dhil-20210728_lab.xml',
   'XBRL TAXONOMY EXTENSION LABEL LINKBASE DOCUMENT',
   'EX-101.LAB',
   '/Archives/edgar/data/909108/000090910821000058/dhil-20210728_lab.xml'),
  ('dhil-20210728_pre.xml',
   'XBRL TAXONOMY EXTENSION PRESENTATION LINKBASE DOCUMENT',
   'EX-101.PRE',
   '/Archives/edgar/data/909108/000090910821000058/dhil-20210728_pre.xml'),
  ('dhil-20210728_htm.xml',
   'EXTRACTED XBRL INSTANCE DOCUMENT',
   'XML',
   '/Archives/edgar/d

In [ ]:
#hide
# uncomment and run to regenerate all library Python files
# from nbdev.export import notebook2script; notebook2script()